In [1]:
import torch
import torch.nn as nn

In [2]:
class convcnp(nn.Module):
    def __init__(self,):
        pass

    def forward(self,):
        pass

In [3]:
import torch
import neuralprocesses.torch as nps
from attrdict import AttrDict

class ConvCNP(torch.nn.Module):
    def __init__(self, dim_x=1, dim_y=1, likelihood="het"):

        super(ConvCNP, self).__init__()
        
        self.model = nps.construct_convgnp(dim_x=dim_x, dim_y=dim_y, likelihood=likelihood)
        
    def forward(self, batch):

        xc, yc = batch.xc, batch.yc
        xt, yt = batch.xt, batch.yt

        # Adjust dimensions to match nps requirements
        xc = xc.permute(0, 2, 1)
        xt = xt.permute(0, 2, 1)
        yc = yc.permute(0, 2, 1)
        yt = yt.permute(0, 2, 1)

        # Calculate loss
        outs = AttrDict()
        outs.tar_ll = nps.loglik(self.model, xc, yc, xt, yt, normalise=True)
        outs.loss = -torch.mean(outs.tar_ll)
        
        # Return in the format expected by the framework
        return outs

    def predict(self, xc, yc, xt, num_samples=1):

        self.eval()
        with torch.no_grad():
            xc = xc.permute(0, 2, 1)
            xt = xt.permute(0, 2, 1)
            yc = yc.permute(0, 2, 1)

            mean, var, _, _ = nps.predict(self.model, xc, yc, xt)

            # Match framework output format
            return AttrDict(
                mean=mean.permute(0, 2, 1),
                scale=var.sqrt().permute(0, 2, 1)
            )

/home/vinayakrana/miniconda3/envs/tnp/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/vinayakrana/miniconda3/envs/tnp/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/home/vinayakrana/miniconda3/envs/tnp/lib/python3.9/site-packages/attrdict/mapping.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping
/home/vinayakrana/miniconda3/envs/tnp/lib/python3.9/site-packages/attrdict/mixins.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprec

In [4]:
import torch
import neuralprocesses.torch as nps
from attrdict import AttrDict

class ConvCNP(torch.nn.Module):
    def __init__(self, dim_x=1, dim_y=1, likelihood="het", device=None):
        super(ConvCNP, self).__init__()

        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = nps.construct_convgnp(dim_x=dim_x, dim_y=dim_y, likelihood=likelihood).to(self.device)
        
    def forward(self, batch):
        xc, yc = batch.xc.to(self.device), batch.yc.to(self.device)
        xt, yt = batch.xt.to(self.device), batch.yt.to(self.device)

        # Adjust dimensions to match nps requirements
        xc, xt = xc.permute(0, 2, 1), xt.permute(0, 2, 1)
        yc, yt = yc.permute(0, 2, 1), yt.permute(0, 2, 1)

        # Calculate loss
        outs = AttrDict()
        outs.tar_ll = nps.loglik(self.model, xc, yc, xt, yt, normalise=True)
        outs.loss = -torch.mean(outs.tar_ll)
        
        return outs

    def predict(self, xc, yc, xt, num_samples=1):
        self.eval()
        with torch.no_grad():
            xc, yc, xt = xc.to(self.device), yc.to(self.device), xt.to(self.device)
            xc, xt = xc.permute(0, 2, 1), xt.permute(0, 2, 1)
            yc = yc.permute(0, 2, 1)

            mean, var, _, _ = nps.predict(self.model, xc, yc, xt)

            return AttrDict(
                mean=mean.permute(0, 2, 1),
                scale=var.sqrt().permute(0, 2, 1)
            )


In [5]:
import torch
from attrdict import AttrDict

# Ensure CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Instantiate model
model = ConvCNP(dim_x=1, dim_y=1, likelihood="het", device=device)

# Generate random batch data
batch_size = 4
num_context = 10
num_target = 15
dim_x = 1
dim_y = 1

batch = AttrDict(
    xc=torch.randn(batch_size, num_context, dim_x).to(device),
    yc=torch.randn(batch_size, num_context, dim_y).to(device),
    xt=torch.randn(batch_size, num_target, dim_x).to(device),
    yt=torch.randn(batch_size, num_target, dim_y).to(device),
)


# Run forward pass
output = model(batch)
print(f"Loss: {output.loss.item()}")

# Run prediction
predictions = model.predict(batch.xc, batch.yc, batch.xt)
print(f"Predicted Mean Shape: {predictions.mean.shape}")
print(f"Predicted Scale Shape: {predictions.scale.shape}")


Using device: cuda
Loss: 1.1139764246647597


AttributeError: module 'lab' has no attribute 'to_torch'